In [4]:
import numpy as np
import csv
import time

np.random.seed(1234)


def randomize(): np.random.seed(time.time()) #현재 시각을 이용해 난수 표현

# 정규분포 난숫값의 평균과 표준편차
#가중치 파라미터를 초기화
RND_MEAN = 0
RND_STD = 0.0030

#학습률 설정
LEARNING_RATE = 0.001

#학습횟수, 미니배치 크기, 중간 보고 주기 등 학습 과정에 관련된 하이퍼파라미터값 설정 ->  train_and_test()로 전달
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset() #데이터 셋 읽어들이기
    init_model() #모델의 파라미터 초기화
    train_and_test(epoch_count, mb_size, report) # 함수를 차례로 호출 후 모델을 생성하고 학습 전체 과정 일괄 처리

#데이터 셋 읽어 들이기
def load_abalone_dataset():
    with open("C:/Users/KimSeongHoon/D_test/chap01/abalone.csv") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt + output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

#파라미터 초기화
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

#학습과 평가 과정을 일괄 실행
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses, accs = [], []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)

        #report 인수로 지정된 보고 주기에 해당하는지 검사
        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch + 1, np.mean(losses), np.mean(accs), acc))

    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))


#학습 및 평가 데이터를 얻을 때 호출하는 세 함수 정의
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count


def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]


def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size * nth:mb_size * (nth + 1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

#train_and_test() 함수가 호출하는 학습 실행 함수와 평가 실행 함수를 정의
def run_train(x, y):
    #순전파 처리
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    #보고용 정확도 계산
    accuracy = eval_accuracy(output, y)

    G_loss = 1.0
    #역전파 처리
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy


def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

#순전파 처리 수행 함수
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

#역전파 처리 수행 함수
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()

    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

#후처리 부분에 대한 순전파 처리 함수
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

#후처리 부분에 대한 역전파 처리 함수
def backprop_postproc(G_loss, diff):
    shape = diff.shape

    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output

#회귀 분석 문제의 특성에 알맞게 정확도 계산 함수
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y) / y))
    return 1 - mdiff


def backprop_postproc_oneline(G_loss, diff):  # backprop_postproc() 대신 사용 가능
    return 2 * diff / np.prod(diff.shape)